## Example 1: Convert Keras App Model to OpenVINO IR

This notebook shows how to convert Keras App Model to OpenVINO IR format.

See list of supported models in Keras-SupportedModelList

Following are the steps
- Install necessary pip packages.
- In AWS Sagemaker environment, remove few unused conda envs. This will ensure we have enought space for OpenVINO docker images.
- Select a model from the supported list
- Run the conversion

### Install necessary pip packages

In [ ]:
!pip3 install --upgrade pip
!pip3 install -r requirements.txt

### Removed few unused conda envs. 

In [ ]:
!bash remove-unused-conda-envs.sh

## Convert Keras App Model to OpenVINO IR

In [ ]:
from ov_utils import create_ir
import os

### Provide below details:

`bucket_name` - S3 bucket name with "sagemaker" as part of the name.
Example: sagemaker-ir-creation

`keras_app_model_name` - Keras app model name. See supported list in Keras-SupportedModelList

`precision_type` - Use either FP32 or FP16

#### NOTE: If the bucket does not exist then it will be created.

#### Example:
```
bucket_name = "sagemaker-ir-creation"
keras_app_model_name = "EfficientNetB0"
precision_type = "FP32"
```

In [ ]:
bucket_name = "sagemaker-ir-creation" # must have sagemaker as part of the name
keras_app_model_name = "Pick-supported-model-name"
precision_type = "FP32" # FP16 also supported

#### Create IR params.

In [ ]:
output_dir = "".join(["./", keras_app_model_name, "-keras-", precision_type])
model_name = "".join([keras_app_model_name])
create_ir_params = {
    "keras_app_model_name": keras_app_model_name,
    "output_dir": output_dir,
    "mo_params": {"data_type": precision_type, "model_name": model_name},
    "bucket_name": bucket_name
}

### Start the IR creation and upload to S3.

In [ ]:
try:
    create_ir(create_ir_params)
    print (f"IR files created for model:{model_name} and the same is uploaded in S3:{bucket_name}")
except Exception as err:
    print(f"FAILED: Please find the error details below:")
    print(err)